<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/LSTM/LSTM01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Treinando o Modelo RNN Para Aprender o Mapeamento das Letras do Alfabeto

Vamos começar projetando um LSTM simples para aprender como prever a próxima letra no alfabeto dado o contexto de apenas um caracter. Vamos considerar o problema como uma coleção aleatória de pares letras de entrada/letras de saída. Como veremos, isso é um problema difícil para modelos LSTM. 

Vamos criar uma rede LSTM com 32 unidades e um único neurônio de saída com função de ativação softmax para fazer previsões. 

In [1]:
# Imports
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

# Random Seed
numpy.random.seed(7)

# Dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Cria mapeamento de caracteres para números inteiros (0-25) e o reverso
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))


char_to_int  

Using TensorFlow backend.


{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [2]:
list(enumerate(alphabet))

[(0, 'A'),
 (1, 'B'),
 (2, 'C'),
 (3, 'D'),
 (4, 'E'),
 (5, 'F'),
 (6, 'G'),
 (7, 'H'),
 (8, 'I'),
 (9, 'J'),
 (10, 'K'),
 (11, 'L'),
 (12, 'M'),
 (13, 'N'),
 (14, 'O'),
 (15, 'P'),
 (16, 'Q'),
 (17, 'R'),
 (18, 'S'),
 (19, 'T'),
 (20, 'U'),
 (21, 'V'),
 (22, 'W'),
 (23, 'X'),
 (24, 'Y'),
 (25, 'Z')]

In [3]:
int_to_char

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z'}

In [4]:
# Prepara o conjunto de dados de pares de entrada/saída codificados como números inteiros
seq_length = 1
dataX = []
dataY = []

for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print (seq_in, '->', seq_out)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [5]:
dataX[:5]

[[0], [1], [2], [3], [4]]

In [6]:
# Reshape de X para [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

# Normalização
X = X / float(len(alphabet))

# One-Hot Encoding para as variáveis de saída
y = np_utils.to_categorical(dataY)

X[:3]

array([[[0.        ]],

       [[0.03846154]],

       [[0.07692308]]])

In [7]:
y[:3]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [8]:
X.shape

(25, 1, 1)

In [9]:
y.shape

(25, 26)

In [10]:
# Fit do Modelo
model = Sequential()
model.add(LSTM(32, input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_1 (Dense)              (None, 26)                858       
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X, y, epochs = 500, batch_size = 1, verbose = 2)

# Performance do Modelo
scores = model.evaluate(X, y, verbose = 0)
print("\nAcurácia do Modelo: %.2f%%" % (scores[1]*100))
print(scores)

Epoch 1/500
 - 2s - loss: 3.2659 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.2582 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 3.2551 - acc: 0.0400
Epoch 4/500
 - 0s - loss: 3.2524 - acc: 0.0400
Epoch 5/500
 - 0s - loss: 3.2495 - acc: 0.0400
Epoch 6/500
 - 0s - loss: 3.2470 - acc: 0.0400
Epoch 7/500
 - 0s - loss: 3.2439 - acc: 0.0400
Epoch 8/500
 - 0s - loss: 3.2411 - acc: 0.0400
Epoch 9/500
 - 0s - loss: 3.2377 - acc: 0.0400
Epoch 10/500
 - 0s - loss: 3.2347 - acc: 0.0400
Epoch 11/500
 - 0s - loss: 3.2311 - acc: 0.0400
Epoch 12/500
 - 0s - loss: 3.2275 - acc: 0.0400
Epoch 13/500
 - 0s - loss: 3.2235 - acc: 0.0400
Epoch 14/500
 - 0s - loss: 3.2202 - acc: 0.0400
Epoch 15/500
 - 0s - loss: 3.2159 - acc: 0.0400
Epoch 16/500
 - 0s - loss: 3.2115 - acc: 0.0400
Epoch 17/500
 - 0s - loss: 3.2064 - acc: 0.0400
Epoch 18/500
 - 0s - loss: 3.2014 - acc: 0.0400
Epoch 19/500
 - 0s - loss: 3.1967 - acc: 0.0400
Epoch 20/500
 - 0s - loss: 3.1908 - acc: 0.0400
Epoch 21/500
 - 0s - loss: 3.1851 - acc: 

In [12]:
# Previsões
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print (seq_in, "->", result)

['A'] -> B
['B'] -> B
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> Y
['X'] -> Z
['Y'] -> Z


Podemos ver que esse problema é de fato difícil para a rede aprender. A razão é que as unidades LSTM não têm nenhum contexto para trabalhar. Cada padrão de entrada-saída é mostrado para a rede em um orden aleatória e o estado da rede é reiniciado após cada padrão (cada lote onde cada lote contém um padrão). Isso é abuso da arquitetura da rede LSTM,
tratando-o como um Perceptron Multilayer padrão. Vamos tentar resolver isso no próximo item.